# 1.Load 
## 1.1 Load libraries

In [1]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.scripts.load_and_save import load_data
from src.utils.exploration_and_clean import explore_column, quick_check_column, clean_metrics


## 1.2 Load data
The dataset is cleaned and filtered the the family of interest.

In [2]:
df = load_data()
df.head()

c:\Users\maudd\Coding\MA3\Ada\ada-2024-project-laambada\src\scripts\load_and_save.py:16: DtypeWarning: Columns (10,11,15,17,20,21,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file_path, compression='zip')


,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,Ligand MW,logP,Charge,Aliphatic OH,Aromatic NH,Ester,Ether,Amide,Ketone,Benzene Ring
0,2880,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK2,Homo sapiens,NaN,16.54,...,509.614,5.3623,0,0,0,0,1,1,0,2
1,2953,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK3,Homo sapiens,NaN,226.9,...,509.614,5.3623,0,0,0,0,1,1,0,2
2,3039,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK2,Homo sapiens,NaN,16.51,...,509.614,5.3623,0,0,0,0,1,1,0,2
3,3041,CS(=O)(=O)c1ccc(cc1)C(=C)n1nc(NC(=O)Nc2ccc(cc2...,InChI=1S/C24H27N7O3S/c1-4-22-27-23(29-31(22)17...,IMNDRGLFDWUUKR-UHFFFAOYSA-N,2061,"US8501936, 90",Tyrosine-protein kinase JAK2,Homo sapiens,NaN,9999,...,493.593,2.8972,0,0,0,0,0,2,0,2
4,3072,CS(=O)(=O)c1ccc(cc1)C(=C)n1nc(NC(=O)Nc2ccc(cc2...,InChI=1S/C24H27N7O3S/c1-4-22-27-23(29-31(22)17...,IMNDRGLFDWUUKR-UHFFFAOYSA-N,2061,"US8501936, 90",Tyrosine-protein kinase JAK3,Homo sapiens,NaN,9999,...,493.593,2.8972,0,0,0,0,0,2,0,2
